In [0]:
import datetime as dt
from pyspark.sql import functions as F
from pyspark.sql.types import *

TODAY = "2025-05-25"   # ← use the same date as in Bronze

BRONZE_RT_PATH = f"dbfs:/bronze/gtfs_rt/{TODAY}"
SILVER_RT_PATH = f"dbfs:/silver/gtfs_rt/{TODAY}"


In [0]:
df_bronze = spark.read.format("delta").load(BRONZE_RT_PATH)
df_bronze.show(3)


In [0]:
df_silver = (
    df_bronze
    .withColumn("event_ts", F.to_timestamp(F.col("timestamp")))
    .withColumn("event_date", F.to_date(F.col("event_ts")))
    .withColumn("location", F.struct(F.col("latitude"), F.col("longitude")))
    .withColumn("processed_at", F.current_timestamp())
    .dropDuplicates(["vehicle_id", "timestamp"])  # avoid duplicate ingests
)


In [0]:
df_silver.write.format("delta").mode("overwrite").save(SILVER_RT_PATH)
print("✓ GTFS-RT Silver transform complete")


In [0]:
spark.read.format("delta").load(SILVER_RT_PATH).show(5)
